In [60]:
import pandas as pd
import numpy as np

In [61]:
!pip install transformers

from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
sinhvien_df = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Raw/01.sinhvien.xlsx")
diem_df = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Raw/02.diem.xlsx")
totnghiep_df = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Raw/14.totnghiep.xlsx")

### Nối bảng dữ liệu

In [ ]:
sinhvien_totnghiep_df = pd.merge(totnghiep_df, sinhvien_df, on='mssv', how='left')

In [ ]:
diem_sinhvien_totnghiep_df = pd.merge(sinhvien_totnghiep_df, diem_df, on='mssv', how='inner')

### Chọn thuộc tính

In [68]:
feature_selection_df = diem_sinhvien_totnghiep_df[['mssv', ' gioitinh', ' khoa', ' hedt', ' mamh', ' sotc', ' diem', ' trangthai', ' namhoc', ' hocky']]

In [69]:
feature_selection_df = feature_selection_df.rename(columns={' gioitinh': 'gioitinh', ' khoa': 'khoa', ' hedt': 'hedt', ' mamh': 'mamh', ' sotc': 'sotc', ' diem': 'diem', ' trangthai': 'trangthai', ' namhoc': 'namhoc', ' hocky': 'hocky'})

### Lọc dữ liệu

In [70]:
filtered_df = feature_selection_df[feature_selection_df['trangthai'].isin([1, 2, 3])]

In [71]:
unique_df = filtered_df.drop_duplicates(subset=['mssv', 'namhoc', 'hocky', 'mamh'])

### Gộp dữ liệu kỳ 3 vào kỳ 2

In [72]:
diem_term_1 = unique_df[unique_df['hocky'] == 1]
diem_term_2 = unique_df[unique_df['hocky'] == 2]
diem_term_3 = unique_df[unique_df['hocky'] == 3]

In [73]:
diem_term_2_3 = pd.merge(diem_term_2, diem_term_3, on=['mssv', 'namhoc', 'mamh'], how='left')[['mssv', 'gioitinh_x', 'khoa_x', 'hedt_x', 'mamh', 'sotc_x', 'diem_x', 'diem_y', 'trangthai_x', 'namhoc', 'hocky_x']]

In [74]:
diem_term_2_3['diem'] = diem_term_2_3.apply(lambda x: x['diem_y'] if not pd.isna(x['diem_y']) else x['diem_x'], axis=1)
diem_term_2_3 = diem_term_2_3.drop(columns=['diem_x', 'diem_y'])
diem_term_2_3 = diem_term_2_3.rename(columns={'gioitinh_x': 'gioitinh', 'khoa_x': 'khoa', 'hedt_x': 'hedt', 'sotc_x': 'sotc', 'trangthai_x': 'trangthai', 'hocky_x': 'hocky'})
diem_df = pd.concat([diem_term_1, diem_term_2_3])

### Mở rộng dữ liệu

In [ ]:
subject_categories = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Augmented/Danh-muc-mon-hoc.xlsx")
subject_description = pd.read_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Augmented/paraphrased_tomtat.xlsx")

In [89]:
filtered_df['mamh'] = filtered_df['mamh'].astype(str)
subject_categories['Mã MH'] = subject_categories['Mã MH'].astype(str)
filtered_df['mamh'] = filtered_df['mamh'].str.strip()
subject_categories['Mã MH'] = subject_categories['Mã MH'].str.strip()

augmented_df = pd.merge(filtered_df, subject_categories, left_on='mamh', right_on='Mã MH', how='left')
augmented_df = augmented_df[list(filtered_df.columns) + ['Đơn vị quản lý chuyên môn', 'Loại MH']]

<ipython-input-89-67598866df23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['mamh'] = filtered_df['mamh'].astype(str)
<ipython-input-89-67598866df23>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['mamh'] = filtered_df['mamh'].str.strip()


In [90]:
augmented_df = augmented_df.rename(columns={'Đơn vị quản lý chuyên môn': 'nganhmh', 'Loại MH': 'loaimh'})

In [91]:
augmented_df = pd.merge(augmented_df, subject_description, on='mamh', how='left')
augmented_df = augmented_df[list(filtered_df.columns) + ['nganhmh', 'loaimh', 'tomtat', 'Paraphrased_Text']]

In [94]:
augmented_df = augmented_df.rename(columns={'Paraphrased_Text': 'paraphased_tomtat'})

### Lọc dữ liệu

In [95]:
filtered_augmented_df = augmented_df[augmented_df['loaimh'] != 'BT']

### Word Embedding dữ liệu tomtat

1. PhoBERT

In [96]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

def phobert_embedding(text):
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
  outputs = model(**inputs)
  embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
  return embeddings

In [97]:
course_paraphased_tomtat = filtered_augmented_df[['mamh', 'paraphased_tomtat']].drop_duplicates(subset=['mamh']).reset_index(drop=True)

In [98]:
PhoBERT_paraphased_tomtat = course_paraphased_tomtat.copy()
PhoBERT_paraphased_tomtat['paraphased_tomtat'] = PhoBERT_paraphased_tomtat['paraphased_tomtat'].apply(lambda x: phobert_embedding(str(x)) if not pd.isna(x) else np.zeros((1, 768)))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [99]:
PhoBERT_paraphased_tomtat['paraphased_tomtat'] = PhoBERT_paraphased_tomtat['paraphased_tomtat'].apply(lambda x: x[0])

In [100]:
embedding_dim = PhoBERT_paraphased_tomtat['paraphased_tomtat'][0].shape[0]
column_names = [f'embedding_dim_{i}' for i in range(embedding_dim)]

embedding_df = pd.DataFrame(PhoBERT_paraphased_tomtat['paraphased_tomtat'].to_list(), columns=column_names)

In [101]:
PhoBERT_paraphased_tomtat = pd.concat([PhoBERT_paraphased_tomtat.drop(columns=['paraphased_tomtat']), embedding_df], axis=1)

### Thuộc tính nâng cao

In [102]:
start_year_df = filtered_augmented_df.groupby('mssv')['namhoc'].min()

In [103]:
start_year_augmented_df = pd.merge(filtered_augmented_df, start_year_df, on='mssv')
start_year_augmented_df = start_year_augmented_df.rename(columns={'namhoc_x': 'namhoc', 'namhoc_y': 'namhoc_batdau'})

In [104]:
term_df = start_year_augmented_df
term_df['sohocky'] = term_df.apply(lambda x: (x['namhoc'] - x['namhoc_batdau']) * 2 + x['hocky'], axis=1)

In [105]:
group_course_df = term_df.copy()
group_course_df['nhomloaimh'] = group_course_df.apply(lambda x: 'CSN' if x['loaimh'] in ['CSNN', 'CSN', 'CNCS'] else 'CN' if x['loaimh'] in ['CNTC', 'CN', 'ĐA'] else 'TN' if x['loaimh'] in ['CĐTN', 'TTTN'] else x['loaimh'], axis=1)

In [106]:
pass_course_df = group_course_df.copy()
pass_course_df['hoanthanh'] = pass_course_df['diem'] >= 5
pass_course_df['hoanthanh'] = pass_course_df['hoanthanh'].astype(int)

In [107]:
group_course_df

,mssv,gioitinh,khoa,hedt,mamh,sotc,diem,trangthai,namhoc,hocky,nganhmh,loaimh,tomtat,paraphased_tomtat,namhoc_batdau,sohocky,nhomloaimh
0,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,1.0,MMT&TT,CQUI,MA003,3.0,6.0,1.0,2013.0,1.0,BMTL,ĐC,Đại số tuyến tính là môn học ở giai đoạn kiến ...,"Môn học: Ma trận, định thức, hệ phương trình t...",2013.0,1.0,ĐC
1,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,1.0,MMT&TT,CQUI,MA001,3.0,6.0,1.0,2013.0,1.0,BMTL,ĐC,NaN,NaN,2013.0,1.0,ĐC
2,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,1.0,MMT&TT,CQUI,PH001,3.0,7.5,1.0,2013.0,1.0,BMTL,ĐC,Đây là môn hoc ̣ ở giai đoan ki ̣ ến thức ...,"Môn học: Khái niệm, phương pháp cơ bản về điện...",2013.0,1.0,ĐC
3,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,1.0,MMT&TT,CQUI,IT001,4.0,7.0,1.0,2013.0,1.0,KHMT,ĐC,và các kỹ năng lập trình thông qua một số bài ...,"Môn học: Kiến thức nền tảng về máy tính, tư d...",2013.0,1.0,ĐC
4,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,1.0,MMT&TT,CQUI,EN001,4.0,6.5,1.0,2013.0,1.0,BMAV,ĐC,NaN,NaN,2013.0,1.0,ĐC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51945,B721B939XPvAibaEXe9Q2McsUcINTscwU/8uZNla,1.0,HTTT,CLC,IT001,4.0,7.7,1.0,2016.0,1.0,KHMT,ĐC,và các kỹ năng lập trình thông qua một số bài ...,"Môn học: Kiến thức nền tảng về máy tính, tư d...",2016.0,1.0,ĐC
51946,B721B939XPvAibaEXe9Q2McsUcINTscwU/8uZNla,1.0,HTTT,CLC,IT009,2.0,7.3,1.0,2016.0,1.0,PĐTĐH,CSNN,NaN,NaN,2016.0,1.0,CSN
51947,B721B939XPvAibaEXe9Q2McsUcINTscwU/8uZNla,1.0,HTTT,CLC,PE001,0.0,9.0,1.0,2016.0,1.0,PĐTĐH,TC,Chương trình môn học giáo dục thể chất (GDTC) ...,"Môn học: Kiến thức, kỹ năng vận động cơ bản, t...",2016.0,1.0,TC
51948,B721B939XPvAibaEXe9Q2McsUcINTscwU/8uZNla,1.0,HTTT,CLC,MA002,3.0,4.2,2.0,2016.0,2.0,BMTL,ĐC,NaN,NaN,2016.0,2.0,ĐC


### Lưu dữ liệu

In [108]:
processed_student_data = pass_course_df[['mssv', 'gioitinh', 'namhoc_batdau', 'khoa', 'hedt']]
processed_score_data = pass_course_df[['mssv', 'mamh', 'nhomloaimh', 'diem', 'trangthai', 'namhoc', 'hocky', 'sohocky', 'hoanthanh']]
processed_course_data = pass_course_df[['mamh', 'sotc', 'nganhmh', 'loaimh', 'nhomloaimh']]
processed_student_data = processed_student_data.drop_duplicates(subset=['mssv'])
processed_course_data = processed_course_data.drop_duplicates(subset=['mamh'])

In [109]:
processed_student_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/processed_student.xlsx")
processed_score_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/processed_score.xlsx")
processed_course_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/processed_course.xlsx")
PhoBERT_paraphased_tomtat.to_csv("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/PhoBERT_paraphased_tomtat.csv")

In [110]:
onehot_encoding_student_data = pd.get_dummies(processed_student_data, columns=['gioitinh', 'khoa', 'hedt'])
onehot_encoding_score_data = pd.get_dummies(processed_score_data, columns=['nhomloaimh', 'trangthai'])
onehot_encoding_course_data = pd.get_dummies(processed_course_data, columns=['nganhmh', 'loaimh', 'nhomloaimh'])

In [111]:
onehot_encoding_student_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/onehot_student.xlsx")
onehot_encoding_score_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/onehot_score.xlsx")
onehot_encoding_course_data.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/onehot_course.xlsx")

### Thống kê môn học

1. Độ phổ biến

In [112]:
subject_popularity = term_df.groupby(['mamh', 'namhoc','sohocky', 'khoa'])['mssv'].count()

In [113]:
subject_popularity = subject_popularity.reset_index().rename(columns={'mssv': 'sosv'})

In [114]:
total_students = term_df.groupby(['namhoc','sohocky', 'khoa'])['mssv'].count().reset_index().rename(columns={'mssv': 'tongsosv'})

In [115]:
subject_popularity = pd.merge(subject_popularity, total_students, on=['namhoc', 'sohocky', 'khoa'], how='left')

In [116]:
subject_popularity['dophobien'] = subject_popularity['sosv'] / subject_popularity['tongsosv']

In [117]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
subject_popularity['dophobien_scaled'] = scaler.fit_transform(subject_popularity[['dophobien']])

In [118]:
subject_popularity.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/subject_popularity.xlsx")

2. Kết quả học tập

In [119]:
subject_score = term_df.groupby(['mamh', 'namhoc', 'sohocky', 'khoa'])['diem'].mean()

In [120]:
subject_score = subject_score.reset_index().rename(columns={'diem': 'diemtb'})

In [121]:
subject_score['dothanhtich'] = subject_score['diemtb'] / 10

In [122]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
subject_score['dothanhtich_scaled'] = scaler.fit_transform(subject_score[['dothanhtich']])

In [123]:
subject_score.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/subject_score.xlsx")

3. Nhóm môn học

In [124]:
group_course = group_course_df.groupby(['khoa', 'namhoc', 'sohocky', 'nhomloaimh'])['mamh'].count() / group_course_df.groupby(['khoa', 'namhoc', 'sohocky', 'nhomloaimh'])['mssv'].nunique()

In [125]:
group_course = group_course.reset_index().rename(columns={0: 'somonhoc'})

In [126]:
group_course.to_excel("/content/drive/MyDrive/Nhóm 5 - DS317.P11/Đồ án mô học/Dataset/Processed/Processed Data With Raw Data/group_course.xlsx")